In [31]:
import json
from pprint import pprint
import pandas as pd
with open('train/cooking_data.json') as data_file:
    data_train = pd.read_json(data_file)
#print (data_train)

In [32]:
from nltk.stem import WordNetLemmatizer
import re
#data_train['process_ingredients'] = [' , '.join(z).strip() for z in data_train['ingredients']]
data_train['process_ingredients'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in data_train['ingredients']] 
#data_train = data_train.drop(columns=['process_ingredients'])
#pprint(data_train.head())
#pprint(data_train)

In [33]:
data_train = data_train.drop(columns=['ingredients'])
#pprint(data_train)

In [34]:
 data_train = data_train.rename(index=str, columns={"process_ingredients": "ingredients"})

In [5]:
pprint (data_train.head())

       cuisine     id                                        ingredients
0        greek  10259  romaine lettuce black olives grape tomatoes ga...
1  southern_us  25693  plain flour ground pepper salt tomato ground b...
2     filipino  20130  egg pepper salt mayonaise cooking oil green ch...
3       indian  22213                     water vegetable oil wheat salt
4       indian  13162  black pepper shallot cornflour cayenne pepper ...


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from nltk.stem import WordNetLemmatizer
corpus = []
total = 39774
for i in range(0, 39774):
    item = re.sub('[^a-zA-Z]',' ',data_train['ingredients'][i])
    item = item.lower()
    item = item.split()
   # print(item)
    #ps = PorterStemmer()
    
    item = [ps.stem(word) for word in item]
    item = ' '.join(item)
    #print (item)
    corpus.append(item)

In [56]:
#Bag of Words Model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder

cv = CountVectorizer()
tf = TfidfTransformer()
#X = cv.fit_transform(corpus).toarray()
X = cv.fit_transform(corpus)
X = tf.fit_transform(X)

print ("X shape -", X.shape)

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(data_train['cuisine'])
print ("y-", set(y))
print ("y-", len(y))

X shape - (39774, 2625)
y- {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
y- 39774


In [58]:
# Splitting the dataset into the Training set and Validation set
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Fitting Multinomial Naive Bayes Model to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

scores = cross_val_score(classifier, X_train, y_train, cv=5)
print('accuracy CV:',scores)

#Predicting for validation set results
y_pred = classifier.predict(X_test)

# # Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm[0])

accuracy CV: [0.66917765 0.65682425 0.6623193  0.65057399 0.65958786]
[22  0  3  1  0  3  0  5  0 20  0  0  0 20  0  0 30  1  2  0]


In [59]:
import numpy as np
for i in range(0,np.size(cm,0)):
    print (cm[i])

[22  0  3  1  0  3  0  5  0 20  0  0  0 20  0  0 30  1  2  0]
[ 0  9  1  0  0 33  0  6  1 28  1  0  0  4  0  0 83  0  0  0]
[  0   0 151   0   0   4   0   3   1  33   0   0   0  21   0   0  57   0
   0   0]
[  0   0   0 519   0   3   0   3   0   4   0   2   0   7   1   0  13   0
   6   3]
[ 0  0  0 57 10  0  0  7  0 17  0  0  0  8  0  0 26  0 15  0]
[  0   3   1   2   0 218   0   3   0 190   0   1   0   6   0   0  98   0
   0   1]
[  0   0   0   0   0   9  72  13   0 127   0   0   0  13   0   0  12   0
   0   0]
[  0   0   1   4   0   0   1 590   0  10   0   0   0  14   1   0  13   0
   2   1]
[ 0  0  0  2  0 16  0  1  8 24  0  0  0  3  0  0 67  0  0  0]
[   0    0    1    0    0   57    2    2    0 1389    0    0    0   25
    1    1   72    0    1    0]
[ 0  1  5  7  0  3  0  8  0 15 13  0  0 20  0  0 49  0  1  0]
[  0   0   1  67   0   4   0  18   0  11   0 149   2   2   0   0  21   0
   0   0]
[ 0  0  0 82  0  0  0  0  0  2  0  9 45  2  0  0  3  0  2  0]
[   0    0    0    4    0  

In [15]:
accuracy = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
precision = cm[1][1]/(cm[1][1] + cm[0][1])
recall = cm[1][1]/ (cm[1][1] + cm[1][0])
f1_score = (2 * precision * recall) / (precision + recall)

In [16]:
print (accuracy)

0.8290598290598291


In [60]:
sum_ = 0
diagonal = 0
for i in range(0,np.size(cm,0)):
    print (cm[i][i], sum(cm[i]))
    diagonal = diagonal + cm[i][i] 
    sum_ = sum_ + sum(cm[i])
    print ("Class - ", i, "Accuracy - ", cm[i][i]/sum(cm[i]))
print ("Avg. Accuracy - ", diagonal/sum_)

22 107
Class -  0 Accuracy -  0.205607476635514
9 166
Class -  1 Accuracy -  0.05421686746987952
151 270
Class -  2 Accuracy -  0.5592592592592592
519 561
Class -  3 Accuracy -  0.9251336898395722
10 140
Class -  4 Accuracy -  0.07142857142857142
218 523
Class -  5 Accuracy -  0.4168260038240918
72 246
Class -  6 Accuracy -  0.2926829268292683
590 637
Class -  7 Accuracy -  0.9262166405023547
8 121
Class -  8 Accuracy -  0.06611570247933884
1389 1551
Class -  9 Accuracy -  0.8955512572533849
13 122
Class -  10 Accuracy -  0.10655737704918032
149 275
Class -  11 Accuracy -  0.5418181818181819
45 145
Class -  12 Accuracy -  0.3103448275862069
1226 1317
Class -  13 Accuracy -  0.9309035687167806
57 161
Class -  14 Accuracy -  0.35403726708074534
2 92
Class -  15 Accuracy -  0.021739130434782608
655 852
Class -  16 Accuracy -  0.7687793427230047
31 199
Class -  17 Accuracy -  0.15577889447236182
237 306
Class -  18 Accuracy -  0.7745098039215687
22 164
Class -  19 Accuracy -  0.13414634146